In [169]:
import pandas as pd
import numpy as np

In [170]:
df = pd.read_csv("other_data.csv")

In [171]:
df

,InvoiceNo,Description,Size,Quantity,UnitPrice,Selling Price,PaymentMode,PaymentDate,PaymentTime
0,1,Dark Green Half Sleeves Polo Mens T-Shirt,XL,1,400,560,Online,2025-03-25,12:31
1,1,Blue Faded Mens Jeans,30,1,900,1200,Online,2025-03-25,12:31
2,2,Gamcha Bath Towel,90,1,140,140,Cash,2025-03-25,15:30
3,3,Macho V Cut Mens Underwear,90,1,125,135,Online,2025-03-25,16:13
4,4,Rectangle Mosquito Net,6x6,1,150,180,Cash,2025-03-25,13:00
...,...,...,...,...,...,...,...,...,...
584,399,Sando Mens Vest,XL,1,260,280,Online,2025-04-26,21:13
585,399,Macho V Cut Mens Underwear,100,1,100,140,Online,2025-04-26,21:13
586,399,Pooja Brand Ladies Bra,38,1,140,155,Online,2025-04-26,21:13
587,399,Boxer Cotton Half Pant,XXL,1,180,250,Online,2025-04-26,21:13


In [172]:
df.loc[211, "Description"] = "Ladies Scarf"

In [193]:
def description_feature_engineering(desc_string):
    # first colour of product
    colors = ["Black", "Blue", "Dark Green", "Brown", "Grey", "Purple", "White", "Red"]
    matched_color = None
    for color in colors:
        if color in desc_string:
            matched_color = color
            desc_string = desc_string.replace(matched_color, "")
            break

    # gender and age group
    matched_gender = None
    matched_agegroup = None
    if "Girl" in desc_string:
        matched_gender = "Female"
        matched_agegroup = "Girl"
        desc_string = desc_string.replace(matched_agegroup, "")
    elif "Ladies" in desc_string:
        matched_gender = "Female"
        matched_agegroup = "Ladies"
        desc_string = desc_string.replace(matched_agegroup, "")
    elif "Boy" in desc_string:
        matched_gender = "Male"
        matched_agegroup = "Boy"
        desc_string = desc_string.replace(matched_agegroup, "")
    elif "Mens" in desc_string:
        matched_gender = "Male"
        matched_agegroup = "Mens"
        desc_string = desc_string.replace(matched_agegroup, "")
    elif "Men" in desc_string:
        matched_gender = "Male"
        matched_agegroup = "Mens"
        desc_string = desc_string.replace(matched_agegroup, "")

    if "Baby" in desc_string:
        matched_agegroup = "Baby"
        desc_string = desc_string.replace(matched_agegroup, "")
    elif "Child" in desc_string or "Kids" in desc_string or "Little" in desc_string:
        matched_agegroup = "Kids"
        desc_string = desc_string.replace(matched_agegroup, "")

    # company string
    companies = ["Adidas", "Ajanta", "Cotton Candy", "ESSA", "Eco Brand", "Evo",
                 "Foggy", "Gem", "Lux Cozi", "Lux Venus", "Macho", "Nike", "Selsia",
                 "Speed", "Stant", "Pooja Brand", "Brazil"]
    matched_company = None
    for company in companies:
        if company in desc_string:
            matched_company = company
            desc_string = desc_string.replace(matched_company, "")
            break

    # category field
    categories = [
        "Handkerchief", "Socks", "Belt", "Cap", "Dungarees", "Frock", "Mask", "Towel", "Bedsheet",
        "T-Shirt", "Shirt", "Track Pant", "Half Pant", "Jeans", "Kurta", "Panties",
         "Leggings", "Plazo", "Underwear", "Inner Slips", "Hand Gloves", "Dupatta", "Blouse", "Bra", "Nighties",
         "Kurti", "Night Dress", "Petticoat", "Stoles", "Scarf", "Blanket", "Vest", "Wallet", "Pillow Cover", "Lungi",
         "Mosquito Net", "Saree", "Sindoor", "Set", "Cycling Pant", "Formal Pant"
    ]
    matched_category = None
    matched_categories = []  # list as we have to handle more than one case
    for category in categories:
        if category in desc_string:
            matched_categories.append(category)

    if len(matched_categories) == 1:
        matched_category = matched_categories[0]
        desc_string = desc_string.replace(matched_categories[0], "")
    elif len(matched_categories) == 2:
        if "Set" in matched_categories:
            matched_category = "Set"
            desc_string = desc_string.replace(matched_category, "")
        elif "Jeans" in matched_categories and "Shirt" in matched_categories:
            matched_category = "Shirt"
            desc_string = desc_string.replace(matched_category, "")
        else:
            # handling cases like "T-Shirt" both Shirt and T-Shirt is matched
            # same for Pant and Panties
            bigger_cate = None
            smll_cate = None
            if len(matched_categories[0]) > len(matched_categories[1]):
                bigger_cate = matched_categories[0]
                smll_cate = matched_categories[1]
            else:
                bigger_cate = matched_categories[1]
                smll_cate = matched_categories[0]

            if smll_cate in bigger_cate:
                matched_category = bigger_cate
                desc_string = desc_string.replace(matched_category, "")
    elif len(matched_categories) > 2:
        if "Set" in matched_categories:
            matched_category = "Set"
            desc_string = desc_string.replace(matched_category, "")

    # details
    cleaned_details = ' '.join(desc_string.split())

    return [matched_color, matched_category, matched_company, cleaned_details, matched_gender, matched_agegroup]

In [174]:
features_df = df['Description'].apply(description_feature_engineering).apply(pd.Series)
features_df.columns = [
    "Product_Color", "Product_Category", "Product_Company", "Product_Details", "Product_Gender", "Product_AgeGroup"
]
df = pd.concat([df, features_df], axis=1)

In [175]:
df[df["Product_Category"].isna()]

,InvoiceNo,Description,Size,Quantity,UnitPrice,Selling Price,PaymentMode,PaymentDate,PaymentTime,Product_Color,Product_Category,Product_Company,Product_Details,Product_Gender,Product_AgeGroup


In [184]:
df[df["Product_Category"] == "Kurta"]

,InvoiceNo,Description,Size,Quantity,UnitPrice,Selling Price,PaymentMode,PaymentDate,PaymentTime,Product_Color,Product_Category,Product_Company,Product_Details,Product_Gender,Product_AgeGroup
33,19,Chikankari Mens Kurta,16 Year’ol,1,650,1050,Cash,2025-03-27,19:30,None,Kurta,None,Chikankari,Male,Mens


In [25]:
categories = df["Product_Category"].sort_values().unique()

In [32]:
df.loc[df["Product_Category"] == "Blanket", "Size"] = "Single"

In [37]:
df.loc[df["Product_Category"] == "Blouse", "Size"] = np.nan

In [42]:
df.loc[364, "Size"] = 32

In [42]:
df.loc[364, "Size"] = 32

In [51]:
df.loc[df["Description"] == "Boxer Cotton Half Pant", "Description"] = "Half Pant"

In [52]:
df.loc[df["Description"] == "Boxer Half Pant", "Description"] = "Half Pant"

In [56]:
df.loc[197, "Selling Price"] = 360

In [63]:
df.loc[df["Description"] == "Mens Jeans", "Description"] = "Denim Dusty Mens Jeans"

In [68]:
df.loc[363, "Description"] = "Rayon Ladies Sleeveless Kurti"

In [77]:
df.loc[[26, 521], "Description"] = "Lycra Leggings"
df.loc[521, "Size"] = "Free Size"

In [86]:
df.loc[518, "Description"] = "Ladies Nighties"

In [92]:
df.loc[543, "Description"] = "ESSA Fair Lady Print IE Ladies Panties"

In [102]:
df.loc[df["Description"] == "Mens Shirt", "Description"] = "Full Sleeves Mens Shirt"

In [103]:
df.loc[[30, 166], "Description"] = "Black Full Sleeves Mens Shirt"

In [108]:
df.loc[[14], "Description"] = "Ankle Length 1 Pair Black Men Socks"

In [114]:
df.loc[df["Description"] == "Bath Towel", "Description"] = "Gamcha Bath Towel"

In [118]:
df.loc[[8], "Description"] = "Track Pant"

In [64]:
categories

array(['Bedsheet', 'Belt', 'Blanket', 'Blouse', 'Bra', 'Cap', 'Dungarees',
       'Dupatta', 'Frock', 'Half Pant', 'Hand Gloves', 'Handkerchief',
       'Inner Slips', 'Jeans', 'Kurta', 'Kurti', 'Leggings', 'Lungi',
       'Mask', 'Mosquito Net', 'Night Dress', 'Nighties', 'Pant',
       'Panties', 'Petticoat', 'Pillow Cover', 'Plazo', 'Saree', 'Scarf',
       'Set', 'Shirt', 'Sindoor', 'Socks', 'Stoles', 'T-Shirt', 'Towel',
       'Track Pant', 'Underwear', 'Vest', 'Wallet'], dtype=object)

In [134]:
df.columns

Index(['InvoiceNo', 'Description', 'Size', 'Quantity', 'UnitPrice',
       'Selling Price', 'PaymentMode', 'PaymentDate', 'PaymentTime',
       'Product_Color', 'Product_Category', 'Product_Company',
       'Product_Details', 'Product_Gender', 'Product_AgeGroup'],
      dtype='object')

In [137]:
df[
    ["InvoiceNo", "Description", "Size", "Quantity", "UnitPrice", "Selling Price", "PaymentMode", "PaymentDate", "PaymentTime"]
].to_csv("final_data.csv", index=False)

In [185]:
final_df = pd.read_csv("final_data.csv")

In [187]:
final_df.loc[33, "Description"] = "Chikankari Mens Kurta Pajama"

In [189]:
final_df.loc[33]

InvoiceNo                                  19
Description      Chikankari Mens Kurta Pajama
Size                               16 Year’ol
Quantity                                    1
UnitPrice                                 650
Selling Price                            1050
PaymentMode                              Cash
PaymentDate                        2025-03-27
PaymentTime                             19:30
Name: 33, dtype: object

In [161]:
final_df.loc[final_df["Description"] == "Formal Black Mens Pant", "Description"] = "Black Mens Formal Pant"

In [163]:
final_df.loc[final_df["Description"] == "Formal Grey Mens Pant", "Description"] = "Grey Mens Formal Pant"

In [165]:
final_df.loc[final_df["Description"] == "Formal Mens Pant", "Description"] = "Mens Formal Pant"

In [177]:
final_df.loc[final_df["Description"] == "Brown Formal Cotton Mens Pant", "Description"] = "Brown Mens Formal Pant"

In [190]:
final_df[
    ["InvoiceNo", "Description", "Size", "Quantity", "UnitPrice", "Selling Price", "PaymentMode", "PaymentDate", "PaymentTime"]
].to_csv("final_data.csv", index=False)

In [198]:
final_df = pd.read_csv("final_data.csv")

In [199]:
features_df = final_df['Description'].apply(description_feature_engineering).apply(pd.Series)
features_df.columns = [
    "Product_Color", "Product_Category", "Product_Company", "Product_Details", "Product_Gender", "Product_AgeGroup"
]
final_df = pd.concat([final_df, features_df], axis=1)

In [200]:
final_df[final_df["Product_Category"] == "Cycling Pant"]

,InvoiceNo,Description,Size,Quantity,UnitPrice,Selling Price,PaymentMode,PaymentDate,PaymentTime,Product_Color,Product_Category,Product_Company,Product_Details,Product_Gender,Product_AgeGroup
490,337,Ladies Cycling Pant,90,1,100,125,Cash,2025-04-20,19:40,None,Cycling Pant,None,,Female,Ladies
